## Importa bibliotecas

In [1]:
import pandas as pd
import numpy as np
import re, sys, os
sys.path.append("../App")
from download_DB import download_db

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

##  Cria conexões e baixa banco de dados

In [3]:
directory = download_db(lista_download = ["vmonitoramentoug" ,"vmonitoramentousina","vmonitoramentoleilao"], force_download=False,queries_path="..//App/Queries")
vmonitoramentoug = pd.read_parquet(f"{directory}vmonitoramentoug.gzip")
vmonitoramentousina = pd.read_parquet(f"{directory}vmonitoramentousina.gzip")
vmonitoramentoleilao = pd.read_parquet(f"{directory}vmonitoramentoleilao.gzip")


******************** Baixando arquivos *********************

Carregando arquivo de log...
vmonitoramentoug já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:06.  Portanto não foi baixado novamente.
vmonitoramentousina já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:10.  Portanto não foi baixado novamente.
vmonitoramentoleilao já foi baixado hoje: Dia: 29/12/22 - Horário: 11:38:16.  Portanto não foi baixado novamente.

************************************************************



### Importa dataframes que foram salvos

In [4]:
# Carrega dataframe com dados do bdworgs
bdworgs = pd.read_parquet("./Intermediarios/bdworgs.gzip") 

### Tratamento de dados


In [5]:
# Carrega tabela de associações
associassoes = pd.read_excel("./Inputs/associassoes.xlsx")
associassoes

,bdworgs,skate
0,Idempreendimento,IdeUsinaOutorga
1,OpTest_R_UG,DatLiberacaoSFGTeste
2,OpCom_Outor_UG,DatUGInicioOpComerOutorgado
3,OpCom_R_UG,DatLiberOpComerRealizado
4,UF,UFUsina
5,NaN,SigTipoGeracao
6,Obras_Real,DatInicioObraRealizado
7,Concretagem_Real,DatConcretagemRealizado
8,Montagem_Real,DatMontagemRealizado
9,Torres_Real,NaN


In [6]:
# Cria dicionário dos nomes das colunas para poder relacionar os nomes das colunas
# Dicionário de colunas bdworgs > skate
dict_bdworgs_skate = dict(zip(associassoes.dropna()["bdworgs"],associassoes.dropna()["skate"]))
# Dicionário de colunas skate > bdworgs
dict_skate_bdworgs = dict(zip(associassoes.dropna()["skate"],associassoes.dropna()["bdworgs"]))

In [7]:
# Colunas carregados do skate
skate_cols = [el for el in associassoes["skate"].dropna()]
# Colunas carregadas do bdworgs
bdworgs_cols =[el for el in associassoes["bdworgs"].dropna()]

# Colunas provenientes do BD skate_UGs
vmonitoramentoug_cols = []

# Colunas provenientes do BD vmonitoramentousina
vmonitoramentousina_col = []

for col in skate_cols:
    if col in vmonitoramentoug.columns:
        vmonitoramentoug_cols.append(col)
    if col in vmonitoramentousina.columns:
        vmonitoramentousina_col.append(col)
        

# Mantém apenas colunas que serão utilizadas        
bdworgs = bdworgs[bdworgs_cols]
vmonitoramentoug = vmonitoramentoug[vmonitoramentoug_cols]
vmonitoramentousina = vmonitoramentousina[vmonitoramentousina_col]
vmonitoramentoleilao = vmonitoramentoleilao[["IdeUsinaOutorga","DatInicioSuprimento"]]


# Em algum momento as informações sobre torres passaram a ser colocadas em montagem 
mask = bdworgs.Montagem_Real.isnull() 
bdworgs.loc[mask,"Montagem_Real"] = bdworgs.loc[mask,"Torres_Real"]

# Renomeia colunas do bdworgs para o padrão do skate
bdworgs.rename(columns=dict_bdworgs_skate,inplace=True)

# A data operação comercial no bdworgs é na verdade a data de obrigação de operação comercial
# Dat_OC_obrigacao = OC_outorga (Fora do ACR) ou OC_suprimento (ACR) 
bdworgs.rename(columns={"DatUGInicioOpComerOutorgado": "Dat_OC_obrigacao"},inplace=True)

bdworgs.sample(5)

,IdeUsinaOutorga,DatLiberacaoSFGTeste,Dat_OC_obrigacao,DatLiberOpComerRealizado,UFUsina,DatInicioObraRealizado,DatConcretagemRealizado,DatMontagemRealizado,Torres_Real,DscComercializacaoEnergia,Num_UG,Pot_UG
2166,29736,2016-10-14,2016-11-23,2016-11-23,RO,2009-12-01,2010-01-01,2010-04-01,nan,ACR,49,75.0
955,32361,2017-08-31,2017-10-01,2017-12-01,PE,2016-11-15,2017-03-31,nan,2017-07-15,ACR,10 e 11,4.6
94,30480,2013-10-16,2013-07-01,2013-11-14,RS,2012-08-01,2012-12-01,2013-09-06,2013-09-15,ACR,8,3.0
1044,33192,2017-08-18,2017-07-01,2017-09-26,MG,2016-09-15,nan,2017-03-09,nan,ACR,1 a 17,16.451614
880,32285,2016-06-11,2015-09-12,2016-07-08,RO,2016-04-01,nan,2016-04-15,nan,ACR,6,1.14


In [8]:
# Ordena dataframe vmonitoramentoleilao por DatInicioSuprimento
vmonitoramentoleilao.sort_values(by="DatInicioSuprimento",inplace=True)

# Remove linhas que possuíam datas de leilão para uma mesma usina e mantém apenas aquelas com a primeira data de 
#de DatInicioSuprimento
vmonitoramentoleilao.drop_duplicates(keep="first",subset= ["IdeUsinaOutorga"],inplace=True)

# Adiciona coluna DatInicioSuprimento ao dataframe vmonitoramentousina
vmonitoramentousina = pd.merge(vmonitoramentousina,
        vmonitoramentoleilao[["DatInicioSuprimento","IdeUsinaOutorga"]],
        on="IdeUsinaOutorga")


vmonitoramentousina[["DscComercializacaoEnergia","DatInicioSuprimento"]].sample(10)

,DscComercializacaoEnergia,DatInicioSuprimento
984,ACR,2024-01-01
2246,Fora do ACR,NaT
2091,ACR,2026-01-01
2877,Fora do ACR,NaT
878,ACR,2017-01-01
2178,Fora do ACR,NaT
943,Fora do ACR,NaT
1395,Fora do ACR,NaT
1752,Fora do ACR,NaT
457,Fora do ACR,NaT


### Adiciona as seguintes  colunas no bdworgs:
- Quant_UG  &rarr; Quantidade de UGs agrupadas
- SigTipoGeracao &rarr; Tipo de geração


In [9]:
def corrige_ug(ug): # Função criada para corrigir nomes das UGs
    ug = ug.replace(" ","")
    ug = ug.replace("e",",")
    ug = ug.replace("-","a")
    ug = re.sub(r",+",r",",ug)
    ug = re.sub(r"([,ae])(0)",r"\1",ug)
    ug = re.sub(r"^0",r"",ug)
    ug = ug.split("a")
    lista = []
    for i in range(len(ug)-1):
        menor = int(ug[i].split(",")[-1])
        maior = int(ug[i+1].split(",")[0])
        list_nums = list(range(menor+1,maior))
        lista += list_nums
    ug = [el.split(",") for el in ug]
    ug = [int(el2) for el1 in ug for el2 in el1]
    ug+= lista
    return (sorted(ug))


# Insere coluna Quant_UG em bdworgs
bdworgs.insert(3,"Quant_UG",bdworgs["Num_UG"].apply(lambda x : len(corrige_ug(x))))
# Remove coluna Num_UG
bdworgs = bdworgs.drop("Num_UG",axis=1)

bdworgs.IdeUsinaOutorga = bdworgs.IdeUsinaOutorga.astype(int)
bdworgs.Quant_UG = bdworgs.Quant_UG.astype(int)
bdworgs.Pot_UG = bdworgs.Pot_UG.astype(float)

ceg_df = pd.read_excel("./Inputs/Relatorio_Outorgas.xlsx")[["Identificador da Usina","Tipo de Geração"]]
dict_id_ceg = dict(zip(ceg_df["Identificador da Usina"],ceg_df["Tipo de Geração"]))
bdworgs["SigTipoGeracao"] = bdworgs["IdeUsinaOutorga"].map(dict_id_ceg)
bdworgs.sample(5)

,IdeUsinaOutorga,DatLiberacaoSFGTeste,Dat_OC_obrigacao,Quant_UG,DatLiberOpComerRealizado,UFUsina,DatInicioObraRealizado,DatConcretagemRealizado,DatMontagemRealizado,Torres_Real,DscComercializacaoEnergia,Pot_UG,SigTipoGeracao
77,30457,2013-12-14,2013-07-01,5,2014-02-19,RS,2012-08-01,2012-11-01,2013-10-29,2013-10-15,ACR,15.0,EOL
579,31425,2016-11-04,2017-01-20,5,2016-11-19,RN,2015-08-15,2016-02-15,nan,2016-09-30,Fora do ACR,10.5,EOL
1945,29453,2010-01-20,2010-02-28,1,2010-03-02,MG,2007-05-14,nan,2008-08-20,nan,ACR,35.0,UHE
1016,33140,2017-08-08,2018-01-01,5,2017-08-29,PI,2016-11-15,2016-12-15,nan,2017-05-30,ACR,10.5,EOL
1197,34074,2018-12-04,2018-11-01,31,2018-12-12,MG,2018-04-15,nan,2018-05-15,nan,ACR,31.0,UFV


### No BD vmonitoramentoug, linhas com todas as colunas iguais tais como IdeUsinaOutorga, DatInicioOpTesteOutorgada... (exceto  MdaPotenciaUnitaria, que não necessariamente tem que ser igual em todas as linhas)  são agrupadas e são criadas duas outras colunas: 
- Quant_UG -> &rarr; Quantidade de UGs agrupadas
- Pot_UG -> &rarr; Soma da potência das UGs que foram agrupadas

#### Entrada: vmonitoramentoug
#### Saída: vmonitoramentoug_grouped

In [10]:
vmonitoramentoug_group_list = [x for x  in vmonitoramentoug.columns.to_list() if x not in ["MdaPotenciaUnitaria"]]
vmonitoramentoug_grouped = vmonitoramentoug.groupby(vmonitoramentoug_group_list,dropna=False).agg(
    Quant_UG = ("MdaPotenciaUnitaria","count"),
    Pot_UG = ("MdaPotenciaUnitaria","sum")
    
).reset_index()

### Realiza o processo acima para o bdworgs:
- Quant_UG -> &rarr; Quantidade de UGs agrupadas. Nesse caso, como o bdworgs, já possuía a coluna Quant_UG, os valores dessa coluna foram somados para os valores agrupados.
- Pot_UG -> &rarr; Soma da potência dos conjuntos de UGs que foram agrupados.

#### Entrada: bdworgs
#### Saída: bdworgs_ug_grouped

In [11]:
dummy_bdworgs = bdworgs.rename(columns={"Quant_UG":"Quant_UG_","Pot_UG":"Pot_UG_"})
dummy_bdworgs_group_list = [x for x  in dummy_bdworgs.columns.to_list() if x not in ["Quant_UG_","Pot_UG_"]]

bdworgs_ug_grouped = dummy_bdworgs.groupby(dummy_bdworgs_group_list,dropna=False).agg(
    Quant_UG = ("Quant_UG_","sum"),
    Pot_UG = ("Pot_UG_","sum")
    
).reset_index() 

### Checa se a quantidade de potência e de usinas nos dataframes iniciais e agrupados são iguais.

In [12]:
print(vmonitoramentoug_grouped.Quant_UG.sum() - vmonitoramentoug.MdaPotenciaUnitaria.count())
print(vmonitoramentoug_grouped.Pot_UG.sum() - vmonitoramentoug.MdaPotenciaUnitaria.sum())
print(bdworgs_ug_grouped.Pot_UG.sum() - bdworgs_ug_grouped.Pot_UG.sum())
print(bdworgs_ug_grouped.Quant_UG.count() - bdworgs_ug_grouped.Quant_UG.count())


0
0.0
0.0
0


### Junta bancos de dados vmonitoramentoug e skate_usina pela coluna IdeUsinaOutorga. Todos os atributos de cada usina são passados para cada UG pertecente a usina.

#### Entrada: vmonitoramentoug_grouped & vmonitoramentousina
#### Saída: skate_merged

In [13]:
skate_merged  = pd.merge(vmonitoramentoug_grouped.drop(["SigTipoGeracao"],axis=1),
        vmonitoramentousina,
        on="IdeUsinaOutorga",
         how="outer")
mask  = skate_merged.DscComercializacaoEnergia == "ACR"
skate_merged.loc[mask,"DatUGInicioOpComerOutorgado"] = skate_merged.DatInicioSuprimento
skate_merged.sample(10)

,IdeUsinaOutorga,DatLiberacaoSFGTeste,DatUGInicioOpComerOutorgado,DatLiberOpComerRealizado,Quant_UG,Pot_UG,UFUsina,SigTipoGeracao,DatInicioObraRealizado,DatConcretagemRealizado,DatMontagemRealizado,DscComercializacaoEnergia,DatInicioSuprimento
1297,35715,2022-11-10,2017-01-01,2022-11-30,2,840.0,PA,UTE,2016-11-15,NaT,2016-12-05,ACR,2017-01-01
2508,44443,NaT,2028-02-13,NaT,11,49995.0,PI,UFV,NaT,NaT,NaT,Fora do ACR,NaT
788,33853,NaT,2023-10-15,NaT,6,37200.0,RN,EOL,2021-10-02,2022-03-15,2022-10-13,Fora do ACR,NaT
3552,49824,NaT,2024-06-01,NaT,5,30000.0,PB,EOL,NaT,NaT,NaT,Fora do ACR,NaT
3042,47370,NaT,2024-07-01,NaT,16,50000.0,MG,UFV,NaT,NaT,NaT,Fora do ACR,NaT
2920,47183,NaT,2023-11-19,NaT,2,8400.0,BA,EOL,NaT,NaT,NaT,Fora do ACR,NaT
2132,40585,2022-10-01,2024-01-01,NaT,11,46200.0,RN,EOL,2021-09-14,2021-11-03,2022-05-15,Fora do ACR,NaT
2838,46881,NaT,2023-05-30,NaT,24,48000.0,BA,UFV,NaT,NaT,NaT,Fora do ACR,NaT
739,33549,2022-12-09,2025-01-01,NaT,1,4500.0,BA,EOL,2022-02-15,2022-04-06,2022-08-17,ACR,2025-01-01
3566,49838,NaT,2024-06-01,NaT,4,24000.0,PB,EOL,NaT,NaT,NaT,Fora do ACR,NaT


In [14]:
# Cria colunas Dat_OC_obrigacao em skate_merged
mask_ACR = skate_merged.DscComercializacaoEnergia == "ACR"
skate_merged.loc[mask_ACR,"Dat_OC_obrigacao"] = skate_merged.DatInicioSuprimento
skate_merged.loc[~mask_ACR,"Dat_OC_obrigacao"] = skate_merged.DatUGInicioOpComerOutorgado

### Converte colunas cujos nomes iniciam em "Dat" para tipo de data. É necessário que as colunas estejam nomeadas no padrão do skate.

In [15]:
colunas_datas = []
for col in set(bdworgs_ug_grouped.columns.to_list()  + skate_merged.columns.to_list()):
    if col[0:3] == "Dat":
        colunas_datas.append(col)
        print(col)
        try:
            skate_merged[col] = pd.to_datetime(skate_merged[col],format="%Y-%m-%d",errors="coerce")
        except KeyError:
            pass
        try:
            bdworgs_ug_grouped[col] = pd.to_datetime(bdworgs_ug_grouped[col],format="%Y-%m-%d",errors="coerce")
        except KeyError:
            pass

DatLiberOpComerRealizado
DatLiberacaoSFGTeste
DatInicioSuprimento
Dat_OC_obrigacao
DatUGInicioOpComerOutorgado
DatMontagemRealizado
DatInicioObraRealizado
DatConcretagemRealizado


### Checa quais colunas não estão presentes em cada um dos dataframes

In [16]:
print("Colunas que estão no skate, porém não no bdworgs:")
for col in skate_merged.columns:
    if col not in bdworgs.columns:
        print(col)
        
print("-"*10)
print("Colunas que estão no bdworgs, porém não no skate:")
for col in bdworgs.columns:
    if col not in skate_merged.columns:
        print(col)

Colunas que estão no skate, porém não no bdworgs:
DatUGInicioOpComerOutorgado
DatInicioSuprimento
----------
Colunas que estão no bdworgs, porém não no skate:
Torres_Real


### Junta BDs skate e bdworgs.
#### Entrada: skate_merged & bdworgs_ug_grouped
#### Saída: skate_merged

In [17]:
bdworgs_ug_grouped["origem"] = "bdworgs"
skate_merged["origem"] = "skate"
bd_combined = pd.concat([bdworgs_ug_grouped,skate_merged],axis=0,ignore_index=True)
#bd_combined = bdworgs_ug_grouped.copy(deep=True)
bd_combined["origem"] = bd_combined["origem"].astype("category")
bd_combined.DscComercializacaoEnergia = bd_combined.DscComercializacaoEnergia.astype("category")
bd_combined.sample(10)

,IdeUsinaOutorga,DatLiberacaoSFGTeste,Dat_OC_obrigacao,DatLiberOpComerRealizado,UFUsina,DatInicioObraRealizado,DatConcretagemRealizado,DatMontagemRealizado,Torres_Real,DscComercializacaoEnergia,SigTipoGeracao,Quant_UG,Pot_UG,origem,DatUGInicioOpComerOutorgado,DatInicioSuprimento
861,30267,2013-04-10,NaT,2013-05-10,SP,NaT,NaT,NaT,nan,Fora do ACR,UTE,1,25.00,bdworgs,NaT,NaT
6518,54908,NaT,2025-07-01,NaT,RN,NaT,NaT,NaT,NaN,Fora do ACR,UFV,12,39180.00,skate,2025-07-01,NaT
3448,35212,2021-05-18,2023-01-01,2022-01-05,RN,2020-10-15,2020-11-15,2021-05-01,NaN,ACR,EOL,1,4200.00,skate,2023-01-01,2023-01-01
6434,54618,NaT,2026-01-01,NaT,MG,NaT,NaT,NaT,NaN,Fora do ACR,UFV,33,49999.95,skate,2026-01-01,NaT
37,1609,2008-05-28,2008-04-20,2008-07-09,SP,2006-10-15,NaT,2006-12-15,nan,ACR,UTE,1,25.00,bdworgs,NaT,NaT
1437,31335,2015-10-28,2015-08-30,2015-12-12,BA,2015-03-13,2015-03-13,NaT,2015-07-15,ACR,EOL,1,2.35,bdworgs,NaT,NaT
3506,35249,2022-05-20,2022-07-22,2022-07-14,PE,2021-03-31,2021-10-29,2022-04-28,NaN,Fora do ACR,EOL,4,17600.00,skate,2022-07-22,NaT
4446,40595,2021-12-17,2023-02-01,2022-02-18,RN,2021-01-18,2021-03-15,2021-09-07,NaN,Fora do ACR,EOL,1,4200.00,skate,2023-02-01,NaT
2428,29177,NaT,2007-12-30,NaT,RS,NaT,NaT,NaT,NaN,Fora do ACR,PCH,1,15000.00,skate,2007-12-30,NaT
3654,35834,2020-02-07,2017-01-01,2021-02-20,AM,2019-06-24,NaT,2019-08-22,NaN,ACR,UTE,14,4928.00,skate,2017-01-01,2017-01-01


In [18]:
bd_combined[["IdeUsinaOutorga","origem","DscComercializacaoEnergia","DatUGInicioOpComerOutorgado","Dat_OC_obrigacao","DatInicioSuprimento"]]

,IdeUsinaOutorga,origem,DscComercializacaoEnergia,DatUGInicioOpComerOutorgado,Dat_OC_obrigacao,DatInicioSuprimento
0,12,bdworgs,ACR,NaT,2007-03-01,NaT
1,12,bdworgs,ACR,NaT,2008-09-01,NaT
2,19,bdworgs,Fora do ACR,NaT,2002-01-01,NaT
3,42,bdworgs,ACR,NaT,2003-11-30,NaT
4,42,bdworgs,ACR,NaT,2005-08-31,NaT
...,...,...,...,...,...,...
6630,70558,skate,Fora do ACR,2027-04-08,2027-04-08,NaT
6631,70559,skate,Fora do ACR,2027-04-08,2027-04-08,NaT
6632,70560,skate,Fora do ACR,2027-04-08,2027-04-08,NaT
6633,70561,skate,Fora do ACR,2027-04-08,2027-04-08,NaT


In [19]:
# Exporta dataframe
bd_combined.to_parquet("./Intermediarios/bd_combined.gzip")